In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('1rex').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [6]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [7]:
lr = LinearRegression(featuresCol='features', labelCol='label',
                     predictionCol='prediction')

In [10]:
lrModel = lr.fit(training)

In [11]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [12]:
lrModel.intercept

0.14228558260358093

In [13]:
training_summary = lrModel.summary

In [14]:
training_summary.rootMeanSquaredError

10.16309157133015

In [15]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [24]:
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [29]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                345|
|   mean| 0.4203174193655331|
| stddev| 10.053974228167895|
|    min|-28.571478869743427|
|    max| 24.290551295953957|
+-------+-------------------+



In [30]:
test_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 156|
|   mean|-0.10453980616293738|
| stddev|   10.90351456884169|
|    min| -26.736207182601724|
|    max|   27.78383192005107|
+-------+--------------------+



In [31]:
correct_model = lr.fit(train_data)

In [32]:
test_results = correct_model.evaluate(test_data)

In [33]:
test_results.rootMeanSquaredError

10.862874487182799

In [34]:
unlabeled_data = test_data.select('features')

In [36]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [37]:
predictions = correct_model.transform(unlabeled_data)

In [40]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  -2.023259153188534|
|(10,[0,1,2,3,4,5,...|-0.46444290676896754|
|(10,[0,1,2,3,4,5,...|   2.040852522201745|
|(10,[0,1,2,3,4,5,...|  0.7515918424848897|
|(10,[0,1,2,3,4,5,...|  0.7609131907830535|
|(10,[0,1,2,3,4,5,...|0.036400494722534094|
|(10,[0,1,2,3,4,5,...| -1.2713740625953736|
|(10,[0,1,2,3,4,5,...| 0.03291409507464621|
|(10,[0,1,2,3,4,5,...|  1.9895482204622768|
|(10,[0,1,2,3,4,5,...| -0.9276265360202149|
|(10,[0,1,2,3,4,5,...|  0.0838853803701777|
|(10,[0,1,2,3,4,5,...|  1.9837137226117123|
|(10,[0,1,2,3,4,5,...| 0.40630971376164565|
|(10,[0,1,2,3,4,5,...|-0.37397722446279585|
|(10,[0,1,2,3,4,5,...|-0.26368111051231174|
|(10,[0,1,2,3,4,5,...|  0.7543024303141881|
|(10,[0,1,2,3,4,5,...| -2.4737085634185316|
|(10,[0,1,2,3,4,5,...| -0.9964193919328224|
|(10,[0,1,2,3,4,5,...|  0.9682405690281075|
|(10,[0,1,2,3,4,5,...|-0.4446755